In [2]:
import asyncio
from scipy import optimize
from tq42.organization import list_all as list_all_organizations
from tq42.project import list_all as list_all_projects
from tq42.experiment import list_all as list_all_experiments
from tq42.channel import Channel, Ask, Tell
from tq42.client import TQ42Client
from tq42.algorithm import AlgorithmProto
from tq42.compute import HardwareProto
import numpy as np
import OptimizationTestFunctions as otf

from tq42.experiment_run import ExperimentRun
from tq42.algorithm import CvaOptParametersProto

In [ ]:
#the main function is declared as async because it contains several await expressions that require it to be asynchronous.
async def main():
    with TQ42Client() as client:
        org_list = list_all_organizations(client=client)
        org_sample = org_list[0]

        proj_list = list_all_projects(client=client, organization_id=org_sample.id)
        proj_sample = proj_list[0]

        exp_list = list_all_experiments(client=client, project_id=proj_sample.id)
        exp_sample = exp_list[0]

        print(f"Running experiment within: \n Org: {org_sample.id} \n Proj: {proj_sample.id} \n Exp: {exp_sample.id}")

        # Create the communication channels which will receive/send the inputs and outputs from/to TetraOpt
        objective_func_channel = await Channel.create(client=client)
        local_opt_channel = await Channel.create(client=client)
        print("objective_func_channel id: ", objective_func_channel)
        print("local_opt_channel id: ", local_opt_channel)

        tetra_opt_params = {
            "dimensionality": 2,
            "iteration_number": 2,
            "maximal_rank": 4,
            "points_number": 1,
            "quantization": False,
            "tolerance": 0.0010000000474974513,
            "lower_limits": [0, 0],
            "upper_limits": [9, 9],
            "grid": [10, 10],
            "objective_function_channel_id": objective_func_channel.id,
            "local_optimizer_channel_id": local_opt_channel.id,
        }

        run = ExperimentRun.create(
            client=client,
            algorithm=AlgorithmProto.TETRA_OPT,
            experiment_id=exp_sample.id,
            compute=HardwareProto.SMALL,
            parameters={'parameters': tetra_opt_params, 'inputs': {}}
        )

        print(f"Experiment run id: {run.id}")

        async def objective_function_callback(ask: Ask) -> Tell:    
            dim = len(ask.headers)
            func = otf.Ackley(dim)
            y = [float(func(np.array(parameter.values))) for parameter in ask.parameters]
            
            tell = Tell(
                parameters=ask.parameters,
                headers=ask.headers,
                results=y
            )
            return tell

        async def local_optimization_function_callback(ask: Ask) -> Tell:
            dim = len(ask.headers)
            func = otf.Ackley(dim)
            results, new_x = [], []

            for parameter in ask.parameters:
                res = optimize.minimize(func, np.array(parameter.values))
                new_x.append({"values": res.x})
                results.append(float(res.fun))

            tell = Tell(
                parameters=ask.parameters,
                headers=ask.headers,
                results=results,
                candidates=new_x
            )
            return tell

        def success():
            print("One async function done!")

        await asyncio.gather(
            objective_func_channel.connect(
                callback=objective_function_callback, finish_callback=success, max_duration_in_sec=None, message_timeout_in_sec=500
            ),
            local_opt_channel.connect(
                callback=local_optimization_function_callback, finish_callback=success, max_duration_in_sec=None, message_timeout_in_sec=500
            )
        )

        print(f"Experiment run result: {run.check()}")
        return run

run = await main()
print(run.poll())